In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
div.container{width:80% !important;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
span.toc-item-num{display:none;}
div.CodeMirror {font-family:Consolas}
div.input {font-family:Consolas}
</style>
"""))

<font size='5' color='red'>ch05. LSTM(RNN)으로 영화평 구분하기</font>
- 5만개 영화 감상평 : 타겟변수(종속변수)로 긍정/부정 구분하기

In [2]:
# 1. 패키지 수입
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from time import time  # time()  # 70.1.1부터 현재까지 몇 초 지났는지

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

from sklearn.metrics import confusion_matrix, f1_score

In [3]:
# 2. 하이퍼 파라미터 설정(이 파라미터를 바꾸면 정확도나 속도에 차이가 남)
MY_WORDS = 10000  # imdb 데이터 안의 단어 수
MY_LENGTH = 80    # 영화평 단어 수 80개만 독립변수로 설정
MY_EMBED = 32     # Embedding 결과 차원
MY_HIDDEN = 64    # LSTM의 units

MY_EPOCH = 10     # 반복할 학습 수(fit)
MY_BATCH = 200    # 배치사이즈(fit)

In [4]:
# 3. 데이터 불러오기
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=MY_WORDS)

In [5]:
print('학습용 입력데이터(독립변수) 모양 : ', X_train.shape)
print('학습용 입력데이터(종속변수) 모양 : ', y_train.shape)
print('학습용 입력데이터 샘플 : ', len(X_train[0]), '-', X_train[0])
print('학습용 출력데이터 샘풀(0:부정/1:긍정) : ', y_train[0])

print('테스트용 입력데이터(독립변수) 모양 : ', X_test.shape)
print('테스트용 입력데이터(종속변수) 모양 : ', y_test.shape)
print('테스트용 입력데이터 샘플 : ', len(X_test[0]), '-', X_test[0])
print('테스트용 출력데이터 샘풀(0:부정/1:긍정) : ', y_test[0])

학습용 입력데이터(독립변수) 모양 :  (25000,)
학습용 입력데이터(종속변수) 모양 :  (25000,)
학습용 입력데이터 샘플 :  218 - [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15,

In [6]:
# 긍정 갯수
print('학습용 데이터의 긍정 갯수 : ', y_train.sum())
print('테스트용 데이터의 긍정 갯수 : ', y_test.sum())

학습용 데이터의 긍정 갯수 :  12500
테스트용 데이터의 긍정 갯수 :  12500


In [7]:
# 4. 영화평 학습용 데이터 처음 10개 길이 출력 함수
def show_length():
    print('첫 10개 영화평의 길이 : ')
    for i in range(10):
        print(f'{i+1}번째 {len(X_train[i])}')
show_length()

첫 10개 영화평의 길이 : 
1번째 218
2번째 189
3번째 141
4번째 550
5번째 147
6번째 43
7번째 123
8번째 562
9번째 233
10번째 130


In [8]:
# [f'{i+1}번째 {len(X_train[i])}' for i in range(10)]

In [9]:
# 5. 문자 단어 -> 정수
word_to_id = imdb.get_word_index()  # 딕셔너리{'문자 단어:key' : 정수:id}
print(word_to_id['movie'])
print(word_to_id['film'])

# 5. 정수 -> 문자 단어
id_to_word = {}  # 딕셔너리{정수:id : '문자 단어:value'}
for key, val in word_to_id.items():
    id_to_word[val] = key
print(id_to_word[17])
print(id_to_word[19])

17
19
movie
film


In [10]:
msg = 'What a wonderful movi'
msg = msg.lower().split()
print([word_to_id.get(m) for m in msg])
print([word_to_id.get(m, -1) for m in msg])
# 1:리뷰시작을 알리는 숫자, 2:문자가 잘려서 잘못 읽어옴
data = [1] + [word_to_id.get(m, -1)+3 for m in msg]
print('원 후기 내용 : ', msg)
print('encoded data : ', data)
print('data 추정 : ', ' '.join([id_to_word.get(d-3, '??') for d in data]))

[48, 3, 386, None]
[48, 3, 386, -1]
원 후기 내용 :  ['what', 'a', 'wonderful', 'movi']
encoded data :  [1, 51, 6, 389, 2]
data 추정 :  ?? what a wonderful ??


In [11]:
# 6. 숫자 영화평 -> 자연어 영화평
def decoding(review_num):
    decoded = []
    for num in review_num:
        word = id_to_word.get(num-3, '???')
        decoded.append(word)
    print(' '.join(decoded))
print(X_train[0])
print(decoding(X_train[0]), y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
??? this film was just brilliant casting location scenery st

In [12]:
print('pad_sequence 작업 전')
show_length()

pad_sequence 작업 전
첫 10개 영화평의 길이 : 
1번째 218
2번째 189
3번째 141
4번째 550
5번째 147
6번째 43
7번째 123
8번째 562
9번째 233
10번째 130


In [13]:
max([len(x) for x in X_train]), min([len(x) for x in X_train])

(2494, 11)

In [14]:
# 7. 모든 영화평의 길이를 동일하게 만들기(score : 75.2%)
X_train = pad_sequences(X_train,  # 2차원
                        padding='post',
                        truncating='post',  # 뒷부분을 자르고 앞부분을 남김
                        maxlen=MY_LENGTH)
X_test = pad_sequences(X_test,
                       padding='post',
                       truncating='post',
                       maxlen=MY_LENGTH)
show_length()

첫 10개 영화평의 길이 : 
1번째 80
2번째 80
3번째 80
4번째 80
5번째 80
6번째 80
7번째 80
8번째 80
9번째 80
10번째 80


In [15]:
# 7. 모든 영화평의 길이를 동일하게 만들기(score : _%)
# X_train = pad_sequences(X_train,  # 2차원
#                         padding='pre',
#                         truncating='pre',  # 뒷부분을 자르고 앞부분을 남김
#                         maxlen=MY_LENGTH)
# X_test = pad_sequences(X_test,
#                        padding='pre',
#                        truncating='pre',
#                        maxlen=MY_LENGTH)
# show_length()

In [16]:
# 8. 모델 생성 전 최종 데이터 shape 확인
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((25000, 80), (25000,), (25000, 80), (25000,))

In [17]:
# 9. 모델 생성 및 구현
model = Sequential()
model.add(Embedding(input_dim=MY_WORDS,
                    output_dim=MY_EMBED, 
                    input_length=MY_LENGTH))
model.add(LSTM(units=MY_HIDDEN,
               input_shape=(MY_LENGTH, MY_EMBED)))
model.add(Dense(units=1, activation='sigmoid'))  # 2진분류
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 80, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 344,897
Trainable params: 344,897
Non-trainable params: 0
_________________________________________________________________


In [18]:
# 10. 학습 환경 설정 및 학습하기
model.compile(loss='binary_crossentropy',  # 이중분류 시 손실함수
              optimizer='adam', metrics=['accuracy'])
begin = time()  # 70.1.1 ~ 현재시점까지의 초
model.fit(x=X_train, y=y_train, epochs=MY_EPOCH, batch_size=MY_BATCH, verbose=1, validation_split=0.2)
end = time()  # 70.1.1 ~ 현재시점까지의 초
print('총 학습 시간 : {:.2f}'.format(end-begin))

Epoch 1/10
100/100 [==============================] - 12s 96ms/step - loss: 0.5843 - accuracy: 0.6607 - val_loss: 0.4474 - val_accuracy: 0.7994
Epoch 2/10
100/100 [==============================] - 9s 87ms/step - loss: 0.3540 - accuracy: 0.8501 - val_loss: 0.4164 - val_accuracy: 0.8184
Epoch 3/10
100/100 [==============================] - 9s 89ms/step - loss: 0.2760 - accuracy: 0.8884 - val_loss: 0.4180 - val_accuracy: 0.8154
Epoch 4/10
100/100 [==============================] - 9s 87ms/step - loss: 0.2344 - accuracy: 0.9141 - val_loss: 0.5258 - val_accuracy: 0.8070
Epoch 5/10
100/100 [==============================] - 9s 89ms/step - loss: 0.2018 - accuracy: 0.9271 - val_loss: 0.4887 - val_accuracy: 0.7908
Epoch 6/10
100/100 [==============================] - 9s 92ms/step - loss: 0.1688 - accuracy: 0.9409 - val_loss: 0.5354 - val_accuracy: 0.7966
Epoch 7/10
100/100 [==============================] - 9s 90ms/step - loss: 0.1500 - accuracy: 0.9467 - val_loss: 0.6879 - val_accuracy: 0.794

In [19]:
# 11. 모델 평가
loss, acc = model.evaluate(x=X_test, y=y_test, verbose=1)
print('test 데이터의 정확도(acc) : ', acc)

782/782 [==============================] - 10s 12ms/step - loss: 0.8070 - accuracy: 0.7562
test 데이터의 정확도(acc) :  0.7562400102615356


In [20]:
# 혼돈행렬
pred = model.predict(X_test)
y_hat = (pred > 0.5).astype(int).reshape(-1)
y_hat

782/782 [==============================] - 9s 11ms/step


array([0, 1, 1, ..., 0, 0, 1])

In [21]:
y_test.shape, y_hat.shape

((25000,), (25000,))

In [22]:
confusion_matrix(y_test, y_hat)

array([[9667, 2833],
       [3261, 9239]], dtype=int64)

In [23]:
import pandas as pd
pd.crosstab(y_test, y_hat)

col_0,0,1
row_0,,
0,9667,2833
1,3261,9239


In [24]:
# accuracy(전체 중 정답을 맞힌 비율) : = (TN+TP) / (TN+FP+FN+TP)
# precision(정밀도, 민감도 - True로 예측한 것 중 맞힌 비율) : = TP / (FP+TP)
# recall(재현율 - 실제값이 True인 것 중 True로 맞힌 비율) : = TP / (FN+TP)

In [25]:
# 12. 모델 사용하기
review = '''What a wonderful movie The role was good the content was good and the sound was good 
There was a disadvantage that I wanted to go to the bathroom in the middle because the running time was long 
The reservation rate is not high so I did well with ease I highly recommend it You won't regret it if you watch it'''
review = review.lower().split()
review = [1] + [word_to_id.get(r, -1)+3 for r in review]  # [1] : 모든 데이터 앞에 1이 존재했으므로 통일시키기 위함
print(len(review), review)
review = np.array(review).reshape(-1, len(review))  # 2차원
review.shape

63 [1, 51, 6, 389, 20, 4, 217, 16, 52, 4, 1500, 16, 52, 5, 4, 481, 16, 52, 50, 16, 6, 23667, 15, 13, 473, 8, 140, 8, 4, 3868, 11, 4, 655, 88, 4, 620, 58, 16, 196, 4, 14802, 967, 9, 24, 312, 38, 13, 122, 73, 19, 3964, 13, 545, 386, 12, 25, 528, 2598, 12, 48, 25, 106, 12]


(1, 63)

In [26]:
input_data = pad_sequences(review,
                           padding='post',
                           maxlen=MY_LENGTH,
                           truncating='post')  # 80개 단어 이상일 경우 뒤부터 자르기
input_data

array([[    1,    51,     6,   389,    20,     4,   217,    16,    52,
            4,  1500,    16,    52,     5,     4,   481,    16,    52,
           50,    16,     6, 23667,    15,    13,   473,     8,   140,
            8,     4,  3868,    11,     4,   655,    88,     4,   620,
           58,    16,   196,     4, 14802,   967,     9,    24,   312,
           38,    13,   122,    73,    19,  3964,    13,   545,   386,
           12,    25,   528,  2598,    12,    48,    25,   106,    12,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0]])

In [30]:
(model.predict(input_data)> 0.5).astype('int8')

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Users\4545\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\4545\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\4545\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\4545\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\4545\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\4545\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\4545\anaconda3\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_o